In [34]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.layers import Normalization, InputLayer, Dense
import tensorflow as tf
import pandas as pd

class RegresionLineal():
    def __init__(self,data):
        self.train = pd.read_csv(data)
        tdata = tf.constant(pd.read_csv(data))
        tdata = tf.cast(tdata,dtype=tf.dtypes.float32)
        self.tdata = tf.random.shuffle(tdata)
        X = tdata[:,3:-1]
        Y = tdata[:,-1]
        Y = tf.expand_dims(Y, axis = -1)
        n = int(0.8 * X.shape[0])
        self.X_train, self.X_test = X[:n], X[n:]
        self.y_train, self.y_test = Y[:n], Y[n:]
    def describe(self):
        self.train.head()
        self.train.describe()
        self.train.info()
        self.train.isnull().sum()
    def normalization(self):
        normalizator = Normalization()
        normalizator.adapt(self.X_train)
        normalizator.adapt(self.y_train)
    def compile(self):
        self.model = self.training()
        self.model.compile(optimizer=Adam(learning_rate=0.01),
            loss=MeanAbsoluteError(),
            metrics=[RootMeanSquaredError()])
        self.model.fit(self.X_train, self.y_train,
                epochs=100, batch_size=32, 
                validation_data=(self.X_test, self.y_test))
    def training(self):
        input_dim = self.X_train.shape[1]
        model = tf.keras.Sequential([
            InputLayer(input_shape=(input_dim,)),
            Dense(1)
        ])
        model.summary()
        return model
    def loss(self):
        loss, rmse = self.model.evaluate(self.X_test, self.y_test, verbose=1)
        print(f"Error medio absoluto: {loss:.2f}")
        print(f"Raíz del error cuadrático medio: {rmse:.2f}")
    def solo(self,vector):
        x = vector[:-1]
        y = vector[-1].numpy()[0][0]
        prediccion = self.modelo.predict(x)[0][0]
        print(f"Valor real: {y:.2f}")
        print(f"Predicción: {prediccion:.2f}")

modelo = RegresionLineal('train.csv')
#modelo.describe()
modelo.normalization()
modelo.compile()
modelo.loss()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 1)                 9         
                                                                 
Total params: 9 (36.00 Byte)
Trainable params: 9 (36.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
25/25 [==============================] - 1s 13ms/step - loss: 320717.6250 - root_mean_squared_error: 343977.3438 - val_loss: 283612.4062 - val_root_mean_squared_error: 309697.7500
Epoch 2/100
25/25 [==============================] - 0s 5ms/step - loss: 295862.8125 - root_mean_squared_error: 323594.9375 - val_loss: 257621.1406 - val_root_mean_squared_error: 289053.7500
Epoch 3/100
25/25 [==============================] - 0s 7ms/step - loss: 271216.9688 - root_mean_squared_error: 304194.1875 - val_loss: 232927.7969 - val_root_mean_squ